In [1]:
import numpy as np

In [2]:
# Example dataset
hours_studied = np.array([1.0, 2.0, 3.0, 4.0, 5.0], dtype=np.float64)  # Feature: Hours studied
exam_score = np.array([2.0, 4.0, 6.0, 8.0, 10.0], dtype=np.float64)    # Target: Exam score


In [3]:
from Pyfhel import Pyfhel

In [4]:
# Assuming w = 2 and b = 0 for demonstration
arr_x = np.array([2.0], dtype=np.float64) 
arr_y = np.array([0.0], dtype=np.float64)




In [5]:
# Feel free to change this number!
n_mults = 1

HE = Pyfhel(key_gen=True, context_params={
    'scheme': 'CKKS',
    'n': 2**14,         # For CKKS, n/2 values can be encoded in a single ciphertext. 
    'scale': 2**30,     # Each multiplication grows the final scale
    'qi_sizes': [60]+ [30]*n_mults +[60] # Number of bits of each prime in the chain. 
                        # Intermediate prime sizes should be close to log2(scale).
                        # One per multiplication! More/higher qi_sizes means bigger 
                        #  ciphertexts and slower ops.
})
HE.relinKeyGen()
print("\nB1. CKKS context generation")
print(f"\t{HE}")


B1. CKKS context generation
	<ckks Pyfhel obj at 0x22ba283d570, [pk:Y, sk:Y, rtk:-, rlk:Y, contx(n=16384, t=0, sec=128, qi=[60, 30, 60], scale=1073741824.0, )]>


In [6]:
ctxt_w_enc  = HE.encryptFrac(arr_x)
ctxt_b_enc  = HE.encryptFrac(arr_y)



In [7]:
hours_studied_enc = [HE.encryptFrac(hours_studied)]
exam_score_enc = [HE.encryptFrac(exam_score)]






In [8]:
print("\nB2. Fixed-point Encoding & Encryption, ")
print("->\tarr_x ", hours_studied,'\n\t==> ctxt_x ', hours_studied_enc)
print("->\tarr_y ", exam_score,'\n\t==> ctxt_y ', exam_score_enc)


B2. Fixed-point Encoding & Encryption, 
->	arr_x  [1. 2. 3. 4. 5.] 
	==> ctxt_x  [<Pyfhel Ciphertext at 0x22ba283d800, scheme=ckks, size=2/2, scale_bits=30, mod_level=0>]
->	arr_y  [ 2.  4.  6.  8. 10.] 
	==> ctxt_y  [<Pyfhel Ciphertext at 0x22ba283d260, scheme=ckks, size=2/2, scale_bits=30, mod_level=0>]


In [13]:
import numpy as np

# Dataset
hours_studied = np.array([1.0, 2.0, 3.0, 4.0, 5.0])  # Feature: Hours studied
exam_score = np.array([2.0, 4.0, 6.0, 8.0, 10.0])    # Target: Exam score
# Model coefficients
w = 2  # Weight
b = 0  # Bias

# Predict exam scores based on hours studied
exam_score_pred = w * hours_studied + b
# Calculate Mean Squared Error
mse = np.mean((exam_score - exam_score_pred)**2)
print("Mean Squared Error:", mse)
print("Actual Exam Scores: ", exam_score)
print("Predicted Exam Scores:", exam_score_pred)



Mean Squared Error: 0.0
Actual Exam Scores:  [ 2.  4.  6.  8. 10.]
Predicted Exam Scores: [ 2.  4.  6.  8. 10.]


In [23]:
# Function to round and truncate results for display
_r = lambda x: np.round(x, decimals=6)

# Example of performing multiple operations with relinearization
print("Performing encrypted prediction with relinearization:")

# Encrypted prediction with relinearization after multiplication
# Note: In a simple linear regression prediction, you typically have one multiplication and one addition
# For demonstration, let's assume we're applying the weight multiple times in a hypothetical scenario
# ctxt_x = hours_studied_enc[2]  # Example: using the first encrypted feature for demonstration

for step in range(1, n_mults + 1):
    ctxt_x = ctxt_x * ctxt_w_enc  # Multiply in-place
    ctxt_x = ~(ctxt_x)  # Relinearize after each multiplication
    # Decrypting for demonstration purposes (in practice, you'd keep it encrypted as long as possible)
    print(f"\tStep {step}: res {_r(HE.decryptFrac(ctxt_x))}")


Performing encrypted prediction with relinearization:


IndexError: list index out of range

In [13]:
# Function to round and truncate results for display
_r = lambda x: np.round(x, decimals=6)

print("Performing encrypted predictions with relinearization for multiple data points:")

# Initialize an empty list to store the encrypted predictions
exam_score_pred_enc = []

# Iterate over each encrypted hour studied
for hour_enc in hours_studied_enc:
    # Apply the encrypted weight (ctxt_w_enc) to the encrypted feature (hour_enc)
    # and add the encrypted bias (ctxt_b_enc) if necessary
    ctxt_pred = hour_enc * ctxt_w_enc  # Encrypted multiplication
    ctxt_pred = ~(ctxt_pred)  # Relinearize after multiplication
    
    # Optionally, add the encrypted bias if your model includes it
    # For simplicity, assuming bias (b) is 0 or already included in weight calculation
    
    # Store the encrypted prediction
    exam_score_pred_enc.append(ctxt_pred)

    # Decrypting for demonstration purposes (in practice, you'd keep it encrypted as long as possible)
    print(f"Encrypted prediction (decrypted for demo): {_r(HE.decryptFrac(ctxt_pred))}")

# Note: This loop demonstrates applying the linear regression model to each encrypted feature
# and includes relinearization after the multiplication operation.


Performing encrypted predictions with relinearization for multiple data points:
Encrypted prediction (decrypted for demo): [ 2.000001e+00 -3.000000e-06  2.000000e-06 ...  0.000000e+00  0.000000e+00
 -0.000000e+00]
